# Trabajando con datos vectoriales

Este ejercitario utiliza las tareas comunes de procesar datos espaciales vectoriales que incluyen:

1. Crear datos geográficos desde texto
2. Visualizar información basica
3. Realizar relacion espacial, por atributos y consultas

Para responder cada pregunta trate crear celdas independientes.

Es necesario importar los siguientes paquetes iniciales para trabajar.

In [11]:
import geopandas # leer y escribir datos espaciales
import matplotlib.pyplot as plt # visualizar informacion
import pandas # para leer y escribir archivos de texto

# Para desplegar paginas web
from IPython.display import IFrame
%matplotlib inline

## Indicaciones

En muchos casos de analisis limpio, el analista ya posee datos preparados para su posterior analisis espacial. Puede que la información esta disponible en formatos conocidos (Ej. shapefiles, raster `tiff`, geopackage `.gpkg` o geojson). sin embargo, es importante entender como crear datos geográficos desde archivos de texto.

A continuación, utilizaremos unos datos de ejemplo de www.infocasas.com.py, con precios y proyectos de departamentos in Asunción, PY. Esta información sirve como ejemplo de datos que son distribuidos sin ningun formato espacial explicito. También se provee los barrios de asunción en poligonos.

_Nota_: 
Si *no encuentra* el archivo `inmuebles.json` asegurese de descargar los archivos `asuncion_barrios.zip` y `inmuebles.zip` necesarios y extrer los mismos en la misma ubicación `datos/`.

## Preparar Datos

### 1. Abrir los datos JSON
Utilice el archivo `datos/inmuebles.json` y abralo con `pandas.read_json` para crear una planilla de calculo de datos en Pandas. 

Puede verificar la ayuda ejecutando con `pandas.read_json?` o en la página https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html. Para leer la informacion en este mismo apartado se puede utilizar en una celda individual.

```python
IFrame("https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html", width=800, height=600)
```


Almacena la informacion en la variable `apto_json` y verifica la informacion para responder las siguientes preguntas:

- ¿Cuántas columnas posee el archivo y describe las más releventas para el analisis espacial precios/ubicación?
- ¿Cuántos apartamentos hay en el archivo?
- ¿Cuáles son las columnas que permiten georeferenciar la información?
- ¿Que tipo de datos poseen las columnas de precio y coordenadas?
- A simple vista, ¿en qué sistema de referencia están las coordenadas?

Utiliza las celdas de abajo para justificar la respuesta con el código necesario.

_Nota_:   
Puede utilizar la función `from_epsg` del módulo de `fiona.crs` para justificar su respuesta en cuanto al sistema de referencias. 

```python
from fiona.crs import from_epsg
```

### 2. Crear geometrias con las coordenadas

Una vez identificadas las columnas con datos geograficos se debe crear una `GeoDataFrame` para trabajar con los datos geograficos y las coordenadas. 

En una variable `geometria` guardar el valor de convertir las coordenadas x e y a tipo puntos de `shapely`.

_Nota_:
Se puede utilizar la funcion utilizada en clase para convertir de cadenas a puntos `Point` utilizando el modulo `shapely.geometry`.

```python
from shapely.geometry import Point
```

### 3. Crear un GeoDataFrame con datos limpios datos

Con las geometrias en la variable `geometria` se puede combinar en una sola variable `inmuebles` la información de análisis por medio del objeto `geopandas.GeoDataFrame` pasando en primer lugar el archivo original y segundo las geometrias, pase los valores en orden:
Ej.

```python
inmuebles = geopandas.GeoDataFrame(apto_json, geometry=geometria)
```

Puede establecer el sistema de coordenadas con el parametro `crs` como tercer parametro o una vez definida la variable `inmuebles` definir la misma.

Para completar este paso se debe:
- a. Crear un GeoDataFrame con las coordenadas convertidas. Verificar la información con el método `.head()`
- b. Haber definido el sistema de coordenadas apropiado para el DataFrame
- c. Guardar el resultado en un archivo shapefile el nombre `"inmuebles.shp"` en la carpeta `datos/` 

_Nota_: Utilice la funcion `to_file` y el parametro `driver` si asi lo requiere con el valor "Esri Shapefile".

### 4. Limpiar datos

Los datos en muchos casos pueden no estar completamente limpios a simple vista. Luego de examinar la información acerca del precio se puede apreciar que los datos se encuentran en cierto formato. 

Ej.

```python
inmuebles["precio"].head(3)
```
El precio tiene un tipo no numérico como cadena, Ej.:

| precio |
|---:|
|U\$S 45.000|
|Gs. 395.000.000|
|Consultar|
|U\$S 125.000|


Pero se debe convertir a un formato numerico, Ej.:

45000

Lo que se requiere:
- Convertir la cadena a número utilizando los métodos de cadena como `.replace()` sobre la serie de la columna `precio`. Prestar atención al valor `"Consultar"` que debe reemplazar por `0`.
- Convertir la cadena a número `float` utilizando el método `.astype()` sobre la serie de la columna `precio`. 
- Crear la columna `precio_final` con los valores limpiados y convertidos. Verificar la informacion con el método `.head()`

_Nota_: 
Todos los métodos de cadena se pueden utilizar directamente sobre una columna de la siguiente manera. Ej.

```python
inmuebles["precio"].str.replace('AAA', 'BBB')
```



### 5. Analizar información

Ahora que tiene los precios finales se puede resumir los datos en base a diferentes monedas. 

Se puede dibujar un histograma con la función `hist` y un filtro de como sigue:

#### Histograma
Crear un histograma por medio de filtros utilizando la moneda del precio. Ej.

```python
filtro = inmuebles["precio"].str.startswith("Gs.")
inmuebles[filtro]["precio_final"].hist(bins=200)
```

Se requiere lo siguiente (crear una celda independiente para las graficas y resultados):
- Cree un filtro en la variable `es_dolar` por medio de la expresion `startswith` sobre la columna `precio`. 
- Cree un filtro en la variable `es_pyg` por medio de la expresion `startswith` sobre la columna `precio`. 
- Dibuje un histograma para la moneda U$S y para la Gs. en base al `precio_final` y verifique la información. 
- ¿Cuál es el departamento con mayor precio en USD dolares y cuanto es el precio final?
- ¿Cuál es el departamento con mayor precio en Gs. Guaraníes y cuanto es el precio final?

_Nota_: 
Puede verificar el valor maximo con el método `describe()` o por medio de un filtro con el método `idxmax()`.

Resumir los datos en dolares
```python
inmuebles[es_dolar]["precio_final"].describe()
```

Ver el mayor precio en moneda Gs.
```python
maximo = inmuebles[es_pyg]["precio_final"].idxmax()
inmuebles.iloc[maximo]
```

### Ver foto del departamento
Verificar la información de la foto ubicado en la columna `img` Ej.

```python
IFrame(src="imagen.png", width=400, height=400)
```

Para ayudar en este ejercicio cree dos variables `maximo_usd` y `maximo_pyg` con los valores de los indices de los departamentos con mayor precio en ambas monedas.

Se requiere lo siguiente (crear una celda independiente para cada pregunta)
- Visualice la imagen del departamento de mayor precio en guaraníes utilizando la funcion `IFrame`.
- Visualice la imagen del departamento de mayor precio en dolares utilizando la funcion `IFrame`.

_Nota_: 
Puede acceder directamente al registro filtrando por la columna `id` o el `precio` si conoce el valor del mismo.

Ej.
```python
# Filtro por ID
inmuebles[inmuebles.id == 186123414]
# Seleccion por precio
inmuebles[inmuebles.precio == "U$S 129.200"]
```


## Resumir Informacion

El ultimo paso seria resumir de manera grafica el rango de precios en USD de los departamentos en los barrios de Asuncion.  Para esto necesitara abrir el shapefile de los barrios en `asuncion_barrios.shp`. La informacion geografica puede ser verificada con la propiedad `.crs` para ver que coincida con el de los inmuebles.

### 1. Abrir los datos SHAPE

Para esto abra el archivo `datos/asuncion_barrios.shp` con la funcion `geopandas.read_file` y verifique la informacion con `.head()` y `.crs`.

Se requiere contestar las siguientes preguntas:
- Crear una variable `asuncion` con la informacion geografica de barrios.
- Verificar que el sistema de referencia de coordenadas sea el mismo que los inmuebles. Que sistema de referencia poseen los poligonos?
- Cuantos barrios posee el archivo con informacion geografica?
- Cual es la columna que identifica el nombre del barrio?

### 2. Reunir los departamentos y barrios

Reunir la informacion por medio de la operacion espacial utilizando la funcion `.sjoin`. Verifique la documentacion para verificar su funcionamiento.

```python
geopandas.sjoin?
```

La funcion `geopandas.sjoin` recibe los parametros:
```python
geopandas.sjoin(
    left_df,
    right_df,
    how='inner',
    op='intersects',
    lsuffix='left',
    rsuffix='right',
)
```
1. El `left_df` datos origen de la operacion `op`
2. El `right_df` datos destino de la operacion `op`
3. La operacion `op` con las opciones "within", "intersect", "contains" 
   - "within": dentro
   - "intersect": intersecta
   - "contains": contiene

Para este caso se debe elegir la operacion mas conveniente de INMUEBLES *DENTRO DE* BARRIOS. Para mas informacion consulte http://geopandas.org/mergingdata.html#spatial-joins. Cual funcion `op="?"` utilizara para este caso.

Se requiere lo siguiente:
- Crea la variable `inmuebles_barrios` con el resultado del JOIN espacial utilizando `inmuebles` y `asuncion`. Solo utilice las columnas "codigo", "barlo_desc", "geometry" para reducir la cantidad de columnas.
- Cuantas columnas resulta en `inmuebles_barrios`?
- Contiene la columna de nombre de barrio?

_Nota_:
Puede solamente seleccionar columnas de interes antes de hacer el join de la siguiente manera:
```python
sub_asuncion = asuncion[["codigo", "barlo_desc", "geometry"]]
```

### 3. Resumir los precios en dolar por barrios

Para poder cuantificar los valores de apartamentos y precios por barrios, se utiliza la funcion `.groupby()` con las columnas que identifican los barrios como "codigo" y "barlo_desc".

Ademas se resumen los datos por medio de la funcion `.agg()` con los valores a clasificar. Se debe guardar el resultado en la variable `resumen`. Para interpretar los valores de precios utilice la expresion:

```python
pandas.options.display.float_format = "{:,.0f}".format
```

Crear la variable resumen del resultado de estas expresiones, notese que se filtran solo los inmuebles en DOLARES:
```python  
agrupar = inmuebles_barrios[es_dolar].groupby(["codigo", "barlo_desc"])
agrupar["codigo", "barlo_desc", "precio_final"].agg(["max","mean","sum","count"])
```

Luego de guardar los datos en la variable `resumen` se debe aplanar las columnas de la siguiente manera:

```python
resumen.columns = resumen.columns.droplevel(0)
```

Interprete los resultados en una celda y debe responder a las preguntas:
- Guardar el resultado en la variable `resumen` y desplegarlo en una celda en filas y columnas
- Asegurarse que `resumen.columns` posea las columnas "mean", "max", "sum" y "count".
- Cual es el barrio con mayor cantidad de inmuebles?
- Cual es el barrio con el mayor precio de inmueble?
- Cual es el barrio cuya suma de precios es mayor?
- Cuanto es el maximo precio promedio?

_Nota_:
Puede responder a las preguntas en la siguiente celda de tipo `Markdown` sin escribir codigo.
Para responder mas rapido con codigo puede utilizar la funcion `.idxmax()`

### 4. Graficar en un mapa el resumen

Para poder graficar el rango de precios de apartamentos por barrios, se debe mezclar el `resumen` con las geometrias de `asuncion` utilizando `.merge()` por medio de la columna en comun "codigo". Verifique la documentacion para verificar su funcionamiento.

```python
resumen.merge?
```

La funcion `.merge()` recibe los parametros:
```python
resumen.merge(
    right,
    how='inner',
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=('_x', '_y'),
    copy=True,
    indicator=False,
    validate=None,
)
```
- El `right` datos destino a hacer el *JOIN* por atributo, en este caso `asuncion`
- El `how` el metodo de mezcla puede ser 'inner' o 'left' como union completa o parcial.
- El `on` la columna atributo a utilizar para la union, en este caso `codigo`.

Vease http://geopandas.org/mergingdata.html#attribute-joins.

*RECUERDE*: Antes de realizar la mezcla aplanar las columnas de la variable `resumen` si no lo hizo en el paso anterior.
```python
resumen.columns = resumen.columns.droplevel(0)
```

Realice el JOIN y guarde en la variable `mezcla`. En este paso se crea un `DataFrame` no geografico, por eso se debe crear como un `GeoDataFrame()` para mostrarlo en un mapa y guardar los resultados en un archivo *shapefile*. 

```python
asuncion_precios = geopandas.GeoDataFrame(mezcla, geometry="geometry", crs=asuncion.crs)
```

Desplegar el mapa resultante con en base a la columna de precio promedio en `precio_final`, notese que se utilizan solo los inmuebles en DOLARES:
```python  
asuncion_precios.plot(figsize=(10,10), legend=True, cmap="Reds", edgecolor="k", column="mean", scheme="Quantiles", k=4)
```
Se utiliza el metodo de clasificacion Cuartiles (`Quantiles`) donde se usan 4 grupos en porcentajes de 25%, 50%, 75%), cada grupo tiene el 25% de los datos. Vease Cuartiles en https://es.wikipedia.org/wiki/Cuantil

Interprete los resultados en una celda y debe responder a las preguntas:
- Guardar el resultado del JOIN en variable `mezcla` teniendo en cuenta la columna adecuada. Cuantos barrios posee los datos de `mezcla`?.
- Grafique el mapa utilizando el precio promedio como columna.
- Grafique el mapa utilizando la cantidad de departamentos como columna.
- Cual es el rango del ultimo Cuartil (>75%) de la columna precio promedio?
- Cual es el rango del ultimo Cuartil (>75%) de la columna cantidad?

_Nota_:
Puede responder a las preguntas en la siguiente celda de tipo `Markdown` sin escribir codigo.